In [1]:
from db_tools.setup import setup
from pyspark.sql.types import StructType,StructField, StringType, IntegerType

In [2]:
from functools import reduce,partial
from pyspark.sql import functions as F

In [3]:
spark = setup()

23/10/29 16:16:17 WARN Utils: Your hostname, luan-Dell-G15-5520 resolves to a loopback address: 127.0.1.1; using 192.168.1.12 instead (on interface wlp0s20f3)
23/10/29 16:16:17 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
23/10/29 16:16:17 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/10/29 16:16:18 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
23/10/29 16:16:18 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.


SparkSession available as "spark"


# OBS: All exercises not present here are basically a copycat of what is found here

# GROUP BY with agregate functions

In [4]:
df1 = spark.sql("""SELECT * FROM payment""")

In [13]:
df1.select(*['customer_id','amount'])\
   .groupBy('customer_id')\
   .agg(F.sum('amount')\
   .alias('total')).orderBy('total',ascending = False).show()

+-----------+------+
|customer_id| total|
+-----------+------+
|        148|211.55|
|        526|208.58|
|        178|194.61|
|        137|191.62|
|        144|189.60|
|        459|183.63|
|        181|167.67|
|        410|167.62|
|        236|166.61|
|        403|162.67|
|        522|161.68|
|        469|158.65|
|        470|157.69|
|        373|156.66|
|        259|154.70|
|        468|154.66|
|        462|152.69|
|        372|152.68|
|        187|151.73|
|        550|151.69|
+-----------+------+
only showing top 20 rows



# GROUP BY with JOIN

In [17]:
df1 = spark.sql("""SELECT customer_id,
                    	first_name || ' ' || last_name AS full_name
                        FROM customer""")
df2 = spark.sql("SELECT customer_id,amount FROM payment")
df_join = df1.join(df2,on = ['customer_id'],how = 'inner')\
             .groupBy('full_name')\
             .agg(F.sum('amount').alias('total'))\
             .orderBy('total',ascending = False)

In [18]:
df_join.show()

+---------------+------+
|      full_name| total|
+---------------+------+
|   Eleanor Hunt|211.55|
|      Karl Seal|208.58|
|  Marion Snyder|194.61|
| Rhonda Kennedy|191.62|
|     Clara Shaw|189.60|
|  Tommy Collazo|183.63|
|    Ana Bradley|167.67|
|    Curtis Irby|167.62|
|    Marcia Dean|166.61|
|       Mike Way|162.67|
|  Arnold Havens|161.68|
|    Wesley Bull|158.65|
|  Gordon Allard|157.69|
|    Louis Leone|156.66|
|    Lena Jensen|154.70|
|       Tim Cary|154.66|
| Warren Sherrod|152.69|
|Steve Mackenzie|152.68|
| Brittany Riley|151.73|
|   Guy Brownlee|151.69|
+---------------+------+
only showing top 20 rows



# GROUP BY with other aggregate functions'

In [19]:
df = spark.sql("SELECT * FROM payment")
df_group_by = df.select('staff_id','payment_id')\
                .groupBy('staff_id')\
                .agg(F.count('payment_id')\
                .alias('payment_no'))
df_group_by.show()

+--------+----------+
|staff_id|payment_no|
+--------+----------+
|       1|      7292|
|       2|      7304|
+--------+----------+



# GROUP BY with multiple fields

In [22]:
df.groupBy(['staff_id','customer_id'])\
  .agg(F.sum('amount')\
  .alias('total'))\
  .orderBy('customer_id').show()

+--------+-----------+-----+
|staff_id|customer_id|total|
+--------+-----------+-----+
|       1|          1|60.85|
|       2|          1|53.85|
|       2|          2|67.88|
|       1|          2|55.86|
|       2|          3|70.88|
|       1|          3|59.88|
|       1|          4|49.88|
|       2|          4|31.90|
|       2|          5|70.79|
|       1|          5|63.86|
|       1|          6|53.85|
|       2|          6|30.90|
|       1|          7|69.84|
|       2|          7|60.88|
|       1|          8|57.86|
|       2|          8|27.91|
|       1|          9|39.88|
|       2|          9|38.92|
|       2|         10|53.88|
|       1|         10|40.88|
+--------+-----------+-----+
only showing top 20 rows



# GROUP BY with DATE

In [26]:
df.select(*[
            'amount',
             F.to_date('payment_date')\
              .alias('paid_date')
        ])\
  .groupBy('paid_date')\
  .agg(F.sum('amount')\
  .alias('total')).show()

+----------+-------+
| paid_date|  total|
+----------+-------+
|2007-03-22|2586.79|
|2007-03-17|2442.16|
|2007-04-27|2673.57|
|2007-03-02|2550.05|
|2007-04-30|5723.89|
|2007-04-26| 347.21|
|2007-04-12|1930.48|
|2007-02-18|1275.98|
|2007-02-14| 116.73|
|2007-03-19|2617.69|
|2007-02-20|1219.09|
|2007-02-19|1290.90|
|2007-04-11|1940.32|
|2007-04-07|1984.28|
|2007-02-17|1188.17|
|2007-04-29|2717.60|
|2007-03-18|2701.76|
|2007-05-14| 514.18|
|2007-03-16| 299.28|
|2007-04-28|2622.73|
+----------+-------+
only showing top 20 rows



In [28]:
df.select('customer_id','amount')\
  .groupBy('customer_id')\
  .agg(F.sum('amount')\
  .alias('total'))\
  .filter(F.col('total')>200).show()

+-----------+------+
|customer_id| total|
+-----------+------+
|        148|211.55|
|        526|208.58|
+-----------+------+

